In [1]:
import requests
import os
from dotenv import load_dotenv
from pathlib import Path
import json
import pandas as pd
# import psycopg2

In [2]:
def load_secrets():
    load_dotenv()
    env_path = Path(".") / ".env"
    load_dotenv(dotenv_path=env_path)

    API_KEY = os.getenv("API_KEY")
    return API_KEY

In [3]:
API_KEY = load_secrets()

In [4]:
url_recent = f"https://airquality.googleapis.com/v1/currentConditions:lookup?key={API_KEY}"

In [5]:
headers = {"Content-Type": "application/json"}

In [6]:
CAPITALS = {
    "Rio Branco": (-9.97499, -67.8243),
    "Maceió": (-9.66599, -35.735),
    "Macapá": (0.0389, -51.0664),
    "Manaus": (-3.11866, -60.0212),
    "Salvador": (-12.9714, -38.5014),
    "Fortaleza": (-3.73186, -38.5267),
    "Brasília": (-15.7797, -47.9297),
    "Vitória": (-20.3155, -40.3128),
    "Goiânia": (-16.6786, -49.2539),
    "São Luís": (-2.52972, -44.3028),
    "Cuiabá": (-15.5989, -56.0949),
    "Campo Grande": (-20.4428, -54.6464),
    "Belo Horizonte": (-19.9167, -43.9345),
    "Belém": (-1.45502, -48.5024),
    "João Pessoa": (-7.115, -34.8631),
    "Curitiba": (-25.4278, -49.2731),
    "Recife": (-8.05, -34.9),
    "Teresina": (-5.08921, -42.8016),
    "Rio de Janeiro": (-22.9068, -43.1729),
    "Natal": (-5.795, -35.2094),
    "Porto Alegre": (-30.0331, -51.23),
    "Porto Velho": (-8.76194, -63.9039),
    "Boa Vista": (2.81972, -60.6733),
    "Florianópolis": (-27.5969, -48.5495),
    "São Paulo": (-23.5505, -46.6333),
    "Aracaju": (-10.9111, -37.0717),
    "Palmas": (-10.1675, -48.3277)
}

In [7]:
df = pd.DataFrame()

In [8]:
capitals_dict = {}
for capital, coords in CAPITALS.items():
    response = requests.post(url=url_recent,
                             json={"location": {
                                 "latitude": coords[0], 
                                 "longitude": coords[1]
                                 }
                            },
                             headers=headers)
    capitals_dict.update({capital : response.text})

In [9]:
print(response.text)

{
  "dateTime": "2025-09-25T21:00:00Z",
  "regionCode": "br",
  "indexes": [
    {
      "code": "uaqi",
      "displayName": "Universal AQI",
      "aqi": 63,
      "aqiDisplay": "63",
      "color": {
        "red": 0.6862745,
        "green": 0.8745098,
        "blue": 0.12941177
      },
      "category": "Good air quality",
      "dominantPollutant": "pm25"
    }
  ]
}



In [10]:
rows = []
for city, data_json in capitals_dict.items():
    city_data = json.loads(data_json)
    idx = city_data.get('indexes')[0]
    rows.append({
        "City": city,
        "Date/Time": city_data.get("dateTime"),
        "Local" : city_data.get("regionCode"),
        "AQI": idx["aqi"],
        "Red": idx.get("color",{}).get("red"),
        "Green": idx.get("color",{}).get("green"),
        "Blue": idx.get("color",{}).get("blue"),
        "Label": idx.get("category"),
        "Dominant Pollutant": idx.get("dominantPollutant")
    })
city_data

{'dateTime': '2025-09-25T21:00:00Z',
 'regionCode': 'br',
 'indexes': [{'code': 'uaqi',
   'displayName': 'Universal AQI',
   'aqi': 63,
   'aqiDisplay': '63',
   'color': {'red': 0.6862745, 'green': 0.8745098, 'blue': 0.12941177},
   'category': 'Good air quality',
   'dominantPollutant': 'pm25'}]}

In [11]:
df = pd.concat([df, pd.DataFrame(rows)], ignore_index=True)

In [12]:
url_history = f"https://airquality.googleapis.com/v1/history:lookup?key={API_KEY}"

capitals_dict_history = {}
for capital, coords in CAPITALS.items():
    response = requests.post(url=url_history,
                             json={
                                 "dateTime": "2025-09-21T15:01:23Z",
                                 "location": {
                                     "latitude": coords[0],
                                     "longitude": coords[1]
                                 },
                            },
                             headers=headers)
    capitals_dict_history.update({capital: response.text})

In [13]:
rows_history = []
for city, data_json in capitals_dict_history.items():
    capitals_dict_history = json.loads(data_json)

    idx = capitals_dict_history.get("hoursInfo")[0].get("indexes")[0]

    rows_history.append({
        "City": city,
        "Date/Time": capitals_dict_history.get("hoursInfo")[0].get("dateTime"),
        "Local" : capitals_dict_history.get("regionCode"),
        "AQI": idx.get("aqi"),
        "Red": idx.get("color",{}).get("red"),
        "Green": idx.get("color",{}).get("green"),
        "Blue": idx.get("color",{}).get("blue"),
        "Label": idx.get("category"),
        "Dominant Pollutant": idx.get("dominantPollutant")
    })

In [14]:
pd.set_option("display.max_rows", None)
df = pd.concat([df, pd.DataFrame(rows_history)], ignore_index=True)
df = df.drop(columns=['Red', 'Green', 'Blue'], axis=1).sort_values(by="City")

In [15]:
order = ['City','AQI','Label','Dominant Pollutant','Local','Date/Time']
df = df[order]
df.head(300)

,City,AQI,Label,Dominant Pollutant,Local,Date/Time
52,Aracaju,70,Good air quality,o3,br,2025-09-21T15:00:00Z
25,Aracaju,76,Good air quality,o3,br,2025-09-25T21:00:00Z
39,Belo Horizonte,70,Good air quality,pm10,br,2025-09-21T15:00:00Z
12,Belo Horizonte,80,Excellent air quality,co,br,2025-09-25T21:00:00Z
40,Belém,64,Good air quality,o3,br,2025-09-21T15:00:00Z
13,Belém,73,Good air quality,o3,br,2025-09-25T21:00:00Z
49,Boa Vista,78,Good air quality,pm10,br,2025-09-21T15:00:00Z
22,Boa Vista,72,Good air quality,o3,br,2025-09-25T21:00:00Z
33,Brasília,59,Moderate air quality,pm10,br,2025-09-21T15:00:00Z
6,Brasília,69,Good air quality,pm10,br,2025-09-25T21:00:00Z


In [16]:
# def preprocess_air_quality():
#     conn = psycopg2.connect(
#         host='Localhost',
#         dbname="meubanco",
#     )sdavadv